In [428]:
# Alphabet conversion
def alphabetEncoding(c):
	if c.isnumeric():
		return int(c)
	if c.isalpha():
		return ord(c.lower()) - ord('a') + 10
	return 36

def alphabetDecoding(num):
	if num < 10:
		return str(num)
	if num < 36:
		return chr(num - 10 + ord('a'))
	return ' '


# Character Conversion
def characterEncoding(group):
	return sum([alphabetEncoding(group[i]) * 37 ** i for i in range(5)])

def characterDecoding(num):
	result = []
	for i in range(5):
		result.append(num % 37)
		num //= 37
	result.reverse()
	return ''.join([alphabetDecoding(result[i]) for i in range(4, -1, -1)])


# Message grouping
def divideMessage(message):
	result = []
	for i in range(0, len(message), 5):
		result.append(message[i:i+5])
	if len(result[-1]) < 5:
		result[-1] += ' ' * (5 - len(result[-1]))
	return result

In [429]:
# Usefull functions for generating prime numbers that will be used in RSA algorithm
import random
from math import log

def rabinMiller(n):
	s = n - 1
	t = 0
	while s & 1 == 0:
		s = s // 2
		t += 1
	k = 0
	while k < 128:
			a = random.randrange(2, n-1)
			#a^s is computationally infeasible.  we need a more intelligent approach
			#v = (a**s)%n
			#python's core math module can do modular exponentiation
			v = pow(a, s, n)
			if v != 1:
				i = 0
				while v != (n - 1):
					if i == t - 1:
						return False
					else:
						i = i + 1
						v = (v**2) % n
			k+=2
	return True

def isPrime(n):
		# lowPrimes is all primes (sans 2, which is covered by the bitwise and operator)
		# under 1000. taking n modulo each lowPrime allows us to remove a huge chunk
		# of composite numbers from our potential pool without resorting to Rabin-Miller
		lowPrimes =   [3,5,7,11,13,17,19,23,29,31,37,41,43,47,53,59,61,67,71,73,79,83,89,97
									,101,103,107,109,113,127,131,137,139,149,151,157,163,167,173,179
									,181,191,193,197,199,211,223,227,229,233,239,241,251,257,263,269
									,271,277,281,283,293,307,311,313,317,331,337,347,349,353,359,367
									,373,379,383,389,397,401,409,419,421,431,433,439,443,449,457,461
									,463,467,479,487,491,499,503,509,521,523,541,547,557,563,569,571
									,577,587,593,599,601,607,613,617,619,631,641,643,647,653,659,661
									,673,677,683,691,701,709,719,727,733,739,743,751,757,761,769,773
									,787,797,809,811,821,823,827,829,839,853,857,859,863,877,881,883
									,887,907,911,919,929,937,941,947,953,967,971,977,983,991,997]
		if (n >= 3):
			if (n & 1 != 0):
				for p in lowPrimes:
					if (n == p):
						return True
					if (n % p == 0):
						return False
				return rabinMiller(n)
		return False

def generateLargePrime(k):
	#k is the desired bit length
	r = 100*(log(k,2)+1) #number of attempts max
	while r > 0:
		#randrange is mersenne twister and is completely deterministic
		#unusable for serious crypto purposes
		n = random.randrange(2**(k-1), 2**(k))
		r -= 1
		if isPrime(n) == True:
			return abs(n)
	return False

In [430]:
# RSA Algorithm
def RSA_Encode(message, e, n):
	groups = divideMessage(message)
	encoded = [characterEncoding(group) for group in groups]
	cipher = [pow(encoded[i], e, n) for i in range(len(encoded))]
	return cipher

def RSA_Decode(cipher, d, n):
	decoded = [pow(cipher[i], d, n) for i in range(len(cipher))]
	message = ''.join([characterDecoding(decoded[i]) for i in range(len(decoded))])
	return message

# Generate RSA key
def tryGenerateKey():
	p = generateLargePrime(512)
	q = generateLargePrime(512)
	while q == p:
		q = generateLargePrime(512)
	n = p * q
	phi = (p - 1) * (q - 1)
	e = generateLargePrime(int(log(phi, 2)))
	d = pow(e, -1, phi)
	return (e, d, n)

def generateKey():
	try:
		return tryGenerateKey()
	except:
		return generateKey()

In [431]:
message = "Hello world"
e, d, n = generateKey()
cipher = RSA_Encode(message, e, n)
returnedMessage = RSA_Decode(cipher, d, n)
print(returnedMessage)


hello world    
